In [ ]:
from dash import Dash, html, dcc, dash_table, Input, Output, State, callback
import pandas as pd
import plotly.express as px

app = Dash(__name__)
default_table = pd.read_csv('../data/cleaned_fantasy_1970.csv')
positions = list(default_table['position'].unique())
positions.append('All')
app.layout = [html.Title('Fantasy Football Dashboard'),
              html.H1('Fantasy Football Dashboard', style={'textAlign': 'center', 'color': 'white'}),
              html.Div(children=[
                  dcc.Dropdown(id='year-dropdown', options=[{'label': i, 'value': i} for i in range(1970, 2024, 1)], value=1970),
                  dcc.Dropdown(id='position-dropdown', options=[{'label': i, 'value': i} for i in positions], value='All'), 
                  dcc.Dropdown(id='columns', options=[{'label': i, 'value': i} for i in default_table.columns], value=['player', 'position', 'ppr_points', 'overall_rank'], multi=True)
              ]),
              html.Div(children=[]),
              html.Div(children=[
                  dash_table.DataTable(id='fantasy-football-table',data=default_table.to_dict('records'), page_size=10, style_cell={'textAlign': 'center'})
              ]),
              html.Div(children=[
                dcc.Graph(id='fantasy-football-graph',figure={})])
            ]

@callback(
    Output('fantasy-football-graph', 'figure'),
    Input('year-dropdown', 'value')
)
def update_graph(selected_year):
    df = pd.read_csv(f'../data/cleaned_fantasy_{selected_year}.csv')
    df100 = df.sort_values('ppr_points',ascending=False).head(100)
    color_dict = {'QB': 'blue', 'RB': 'green', 'WR': 'red', 'TE': 'purple'}
    fig = px.pie(df100, df100['position'], title = f'Top 100 Fantasy Football Players by Position in {selected_year} (PPR Points)', 
                 color='position', color_discrete_map=color_dict)
    return fig

@callback(
    Output('fantasy-football-table', 'data'),
    Input('position-dropdown', 'value'),
    Input('year-dropdown', 'value'),
    Input('columns', 'value')
)
def update_table(selected_position, selected_year, columns):
    df = pd.read_csv(f'../data/cleaned_fantasy_{selected_year}.csv')
    df = df.filter(columns).sort_values('ppr_points',ascending=False)
    if selected_position == 'All':
        return df.to_dict('records')
    else:
        return df[df['position'] == selected_position].to_dict('records')

if __name__ == '__main__':
    app.run(debug=True)

Ideas:
1. Add scoring format dropdown
2. Chart the percentage of Top 100 players by position over time
3. Create custom chart type that displays where on the field players threw/caught the ball, encode catch percentage as size or gradient